In [1]:
import numpy as np
import math
from datetime import datetime

class HeuristicScorer:
    def __init__(self):
        # Configuration Hyperparameters
        self.weights = {
            'Full-Time': 1.0,
            'Freelance': 0.8,
            'Internship': 0.7
        }
        self.duration_scale_k = 24.0  # Saturation point (months)
        self.recency_lambda = 0.15    # Decay rate (yearly)
        self.impact_threshold = 5.0   # Max score cap

    def calculate_score(self, experience_entry, bert_relevance_score):
        """
        Calculates the S(e) for a single experience entry.
        """
        # 1. Type Weight (Wt)
        w_type = self.weights.get(experience_entry['type'], 0.5)

        # 2. Duration Score (D) - Tanh Normalization
        months = experience_entry['duration_months']
        d_norm = np.tanh(months / self.duration_scale_k)

        # 3. Recency Score (C) - Exponential Decay
        years_elapsed = self._get_years_elapsed(experience_entry['end_date'])
        c_decay = math.exp(-self.recency_lambda * years_elapsed)

        # 4. Impact Score (I) - Heuristic Keyword Counting
        # Assuming you pre-processed verb_count and metric_count
        v_score = experience_entry['verb_count'] * 0.5
        n_score = experience_entry['metric_count'] * 1.0
        i_score = min(1.0, (v_score + n_score) / self.impact_threshold)

        # 5. Semantic Relevance (R)
        r_bert = bert_relevance_score

        # Final Formula
        final_score = w_type * d_norm * c_decay * i_score * r_bert
        
        return final_score

    def _get_years_elapsed(self, end_date_str):
        if end_date_str.lower() == 'present':
            return 0.0
        try:
            # Assumes format "YYYY-MM"
            end_date = datetime.strptime(end_date_str, "%Y-%m")
            current_date = datetime.now()
            delta_days = (current_date - end_date).days
            return max(0, delta_days / 365.0)
        except:
            return 5.0 # Fallback penalty for bad dates